In [1]:
from scipy.integrate import odeint
from IPython.display import display

import ipywidgets as wg
import plotly.graph_objects as go
import plotly.express as pl
import plotly.io as pio

import numpy as np
import pandas as pd
import json

with open('theme_template.json', 'r') as template:
    style = json.load(template)
style["layout"]["height"] = 2*61 * (600 / 158.75)
style["layout"]["width"] = 2*70 * (600 / 158.75)

pio.templates["paper"] = go.layout.Template(
    data=style["data"],
    layout=style["layout"]
)
pio.templates.default = "simple_white+paper"


In [11]:
def electric_heater(T, t, u):
    Tr, TL = T
    # Parameters
    # U = global heat transfer coefficient
    # U_loss = heat losses to air
    # A = area of heat Transfer
    # T_amb = ambient temperature
    # L_mass = mass of the process liquid
    # gain = gain

    UA_loss = 0.1049  # W / m^2 s
    UA = 0.6539  # W / m^2 s
    UA_loss1 = 0.09409
    gain = 0.1319
    T_amb = 24
    L_mass = 1500
    Cp = 4.186  # J / g K
    mcpr = 153

    Qt = UA * (Tr - TL)
    Q_loss = UA_loss * (Tr - T_amb)
    Q_loss1 = UA_loss1 * (TL - T_amb)

    dTLdt = (Qt - Q_loss1) / (L_mass * Cp)
    dTrdt = (gain * u - (Qt + Q_loss)) / mcpr

    return [dTrdt, dTLdt]
    
def simulate(kp, ki, kd, set_point):

    #                  T_liquid, T_resistor
    initial_conditions = [25, 25]
    simulation_results = pd.DataFrame(columns=["Time", "T_resistance", "T_liquid", "u"])
    simulation_results.loc[0, "Time"] = 0
    simulation_results.loc[0, "T_liquid"] = initial_conditions[0]
    simulation_results.loc[0, "T_resistance"] = initial_conditions[1]
    simulation_results.loc[0, "u"] = 0

    N = int(np.ceil(5*3600/2))
    time_final = 5*3600
    time = np.linspace(0, time_final, N)
    I = np.zeros(N)
    error = np.zeros(N)
    u = np.zeros(N)

    for i in range(1, N):
        dt = time[i] - time[i - 1]
        time_interval = [time[i - 1], time[i]]
        solution = odeint(
            electric_heater, initial_conditions, time_interval, args=(u[i-1],)
        )

        error[i] = set_point - solution[-1, 1]
        P = kp * error[i]
        I[i] = I[i - 1] + ki * error[i] * dt
        D = kd * (error[i] - error[i - 1]) / dt
        u[i] = np.ceil(P + I[i] + D) 
        if u[i] > 255:
            u[i] = 255
        if u[i] < 0:
            u[i] = 0

        simulation_results.loc[i, "Time"] = time[i]
        simulation_results.loc[i, "T_resistance"] = solution[-1, 0]
        simulation_results.loc[i, "T_liquid"] = solution[-1, 1]
        simulation_results.loc[i, "u"] = u[i]
        initial_conditions = [solution[-1, 0], solution[-1, 1]]
    
    fig_temps = pl.line(simulation_results, x="Time", y=["T_liquid", "T_resistance"])
    fig_control = pl.line(simulation_results, x="Time", y="u")
    fig_temps.show()
    fig_control.show()


In [12]:

kp_wg = wg.BoundedFloatText(value=1, max=10000)
ki_wg = wg.BoundedFloatText(value=0, max=10000)
kd_wg = wg.BoundedFloatText(value=0, max=10000)
sp_wg = wg.BoundedFloatText(value=50, max=10000)
wg.interact(
    simulate,
    kp=kp_wg,
    ki=ki_wg,
    kd=kd_wg,
    set_point=sp_wg
)

interactive(children=(BoundedFloatText(value=1.0, description='kp', max=10000.0), BoundedFloatText(value=0.0, …

<function __main__.simulate(kp, ki, kd, set_point)>